<a href="https://colab.research.google.com/github/TitasDas/CNN-feature-extraction/blob/main/transfer_learning_vgg19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle

from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 

In [ ]:
input_images = []
